# 2주차 복습과제 - 교차검증

In [47]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score #Confusion matrix 수업 때 진행할 예정
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score#임포트
import numpy as np
import pandas as pd

# 0. Data Load

In [48]:
fold_iris = load_iris()
features = fold_iris.data
label= fold_iris.target

# 1. Train/Test split
- 기본 모델로 나눠서 실제 과적합이 어떤 식으로 발생하는지 확인!

In [49]:
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=111)

In [50]:
model = DecisionTreeClassifier()
model_sp1 = DecisionTreeClassifier(random_state=111, max_depth=3)
model.fit(X_train, y_train) #모델 학습
model_sp1.fit(X_train, y_train) #모델 학습 - 하이퍼 파라미터 존재하는 모델
pred_train = model.predict(X_train) #train 예측
pred_test = model.predict(X_test) # test예측
pred_train_sp = model_sp1.predict(X_train) #train 예측 - 하이퍼 파라미터 존재하는 모델
pred_test_sp = model_sp1.predict(X_test) # test예측 - 하이퍼 파라미터 존재하는 모델

In [51]:
# train이 1.0 > test가 0.9 => 과적합 문제 발생!
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

1.0
0.9


In [52]:
print(accuracy_score(y_train, pred_train_sp))
print(accuracy_score(y_test, pred_test_sp))

0.9916666666666667
0.9


- max_depth =3으로 진행하면 기존 train 데이터 정확도는 1.0 에서 0.9916666666666667 정확도가 줄었다.
- DT : 가지치기 형식으로 뻗어나가는 구조. 데이터를 나눠가면서 분리하는데 max_depth(최대깊이)를 최소화(3)로 줄이니까 train 정확도가 줄었다.
- 물론 Iris 데이터가 너무 작기 때문에 그럴 수도 있지만
- 최대 깊이 지정 X : 데이터의 모든 것을 하나하나 분리하고 학습
- 최대 깊이 지정 O : 분리하는 과정을 3개까지만 하고 분리 멈춤 -> 아직 분리할 것들이 남았지만 그 부분은 학습하지 않음
- 기존에 1이 아닌 정확도가 조금 줄게  되었고, 이 부분은 과적합을 막기 위해서 하이퍼파라미터를 튜닝해서 과적합을 줄이고 일반화를 하기 위해 진행한 것!

# 2. 교차검증(CV- Cross Validation)을 통해서 과적합이 나눠지는지?

In [53]:
from sklearn.model_selection import KFold
kfold= KFold(n_splits=5) #데이터셋을 분리하는 것
cv_accuracy_train=[]
cv_accuracy_test=[]
kf_model = DecisionTreeClassifier(random_state=111, max_depth=3)

In [54]:
kfold

KFold(n_splits=5, random_state=None, shuffle=False)

In [55]:
label

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [63]:
n_iter = 0 #5번 진행하니 초깃값 설정

for train_idx, test_idx in kfold.split(features):

    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]

    #학습을 위해서 dt배우기
    kf_model.fit(X_train, y_train)

    #예측
    kf_pred_train = kf_model.predict(X_train)
    df_pred_test = kf_model.predict(X_test)

    #정확도를 5번 측정하면서 값에 넣을 것이니
    n_iter +=1
    accuracy_train = np.round(accuracy_score(y_train, kf_pred_train),4) # train
    accuracy_test = np.round(accuracy_score(y_test, df_pred_test),4) # test

    print('\n{} 번 train 교차 검증 정확도 :{} ,test 교차 검증 정확도 :{} '.format(n_iter,accuracy_train, accuracy_test))

    cv_accuracy_train.append(accuracy_train)
    cv_accuracy_test.append(accuracy_test)
print('')
print('train 평균 정확도',np.mean(cv_accuracy_train))
print('test 평균 정확도',np.mean(cv_accuracy_test))


1 번 train 교차 검증 정확도 :0.975 ,test 교차 검증 정확도 :1.0 

2 번 train 교차 검증 정확도 :0.975 ,test 교차 검증 정확도 :0.9667 

3 번 train 교차 검증 정확도 :0.9917 ,test 교차 검증 정확도 :0.8333 

4 번 train 교차 검증 정확도 :0.9833 ,test 교차 검증 정확도 :0.9333 

5 번 train 교차 검증 정확도 :0.975 ,test 교차 검증 정확도 :0.7 

train 평균 정확도 0.9799999999999999
test 평균 정확도 0.8866599999999998


- 반복횟수를 거듭할 수록 train의 정확도는 점차 감소
- 기존에 하이퍼 파라미터가 없는 경우는 다 1로 나옴
- 결과적으로는 test 평균 정확도 0.9로 떨어짐
- train은 0.975인데 test는 1이 나왔다? 이건 어떻게 해석??!!

# 3. Skf 모델 진행
- 과적합을 막기 위해서 Skf 방식 사용 가능
- 모델링에서 과적합을 막기 위해 하이퍼파라미터 추가 가능
- kfold의 문제점은 ?
  - 정답라벨이 0,1,2 총 3개 -> KFold는 랜덤하게 데이터를 추출하기 때문에 정답 비중이 고르게 분포하지 않을 수 있다.
  - 그러다보니 정답레이블이 불균형일 수 있다. 불균형 또는 다중분류인 경우는 Skf를 통해 정답 라벨 불균형을 해소해준다!

In [58]:
label
# 0,1,2로 3개로 분류

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [59]:
from sklearn.model_selection import StratifiedKFold
skf_iris = StratifiedKFold(n_splits=3)
cnt_iter=0

In [60]:
skf_iris

StratifiedKFold(n_splits=3, random_state=None, shuffle=False)

In [68]:
n_iter=0

skf_cv_accuracy_train = []
skf_cv_accuracy_test = []
skf_model = DecisionTreeClassifier(random_state=111, max_depth=3)

for train_idx, test_idx in skf_iris.split(features, label):
    X_train, X_test = features[train_idx], features[test_idx]
    y_train, y_test = label[train_idx], label[test_idx]

    #학습을 위해서 dt배우기
    skf_model.fit(X_train, y_train)
    #예측
    skf_pred_train = skf_model.predict(X_train)
    skf_pred_test = skf_model.predict(X_test)

    #정확도를 5번 측정하면서 값에 넣을 것이니
    n_iter +=1
    accuracy_train = np.round(accuracy_score(y_train, skf_pred_train),4)
    accuracy_test = np.round(accuracy_score(y_test, skf_pred_test),4)

    print('\n{} 번 train 교차 검증 정확도 :{} ,test 교차 검증 정확도 :{} '.format(n_iter,accuracy_train, accuracy_test))

    skf_cv_accuracy_train.append(accuracy_train)
    skf_cv_accuracy_test.append(accuracy_test)
print('')
print('train 평균 정확도',np.mean(skf_cv_accuracy_train))
print('test 평균 정확도',np.mean(skf_cv_accuracy_test))


1 번 train 교차 검증 정확도 :0.95 ,test 교차 검증 정확도 :0.98 

2 번 train 교차 검증 정확도 :0.99 ,test 교차 검증 정확도 :0.94 

3 번 train 교차 검증 정확도 :0.97 ,test 교차 검증 정확도 :0.96 

train 평균 정확도 0.9700000000000001
test 평균 정확도 0.96


- train에 대해서 과적합이 되는 것은 iris 데이터가 작아서 그런건가?
- 최소 타이타닉 데이터 정도 되면 달라질까?

# 5. KFold, SKf 정리
## Kfold
- 학습 시에 0,1,2로 구성되어 있는 정답데이터가 고르게 분포 되어 있더라도, 랜덤하게 추출하다 보니 0,1만 있는 데이터, 0만 있는 데이터, 0,2만 있는 데이터로 추출 될 가능성이 있다.
- 추출의 비중도 0이 10개, 1이 5개 이렇게
- 이런 식으로 학습할 데이터의 라벨이 달라지면 정확도의 문제가 올 수 있다.

## Skfold
- 학습시에 정답레이블의 불균형을 맞춰준다.
- 0,1,2 라벨로 구성되어 있으면 0,1,2이 각각 10개씩 처럼 비중도 맞춰서 학습하게 된다.
- 정답레이블에 대한 불균형은 항상 모델링을 진행할 때 확인해야한다!
- iris 데이터는 150개 정도라서 다른 데이터 set을 통해서 과제 진행해보자!

# 6. 실제 레이블 개수 & 비율 확인

In [70]:
import pandas as pd
iris=load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label'] = iris.target

In [71]:
## KFold
kfold= KFold(n_splits=5)
n_iter = 0

for train_idx, test_idx in kfold.split(iris_df):
    n_iter +=1
    label_train = iris_df['label'].iloc[train_idx]
    label_test = iris_df['label'].iloc[test_idx]
    print('학습 정답 레이블', label_train.value_counts())
    print('학습 정답 레이블', label_test.value_counts())

학습 정답 레이블 1    50
2    50
0    20
Name: label, dtype: int64
학습 정답 레이블 0    30
Name: label, dtype: int64
학습 정답 레이블 2    50
1    40
0    30
Name: label, dtype: int64
학습 정답 레이블 0    20
1    10
Name: label, dtype: int64
학습 정답 레이블 0    50
2    50
1    20
Name: label, dtype: int64
학습 정답 레이블 1    30
Name: label, dtype: int64
학습 정답 레이블 0    50
1    40
2    30
Name: label, dtype: int64
학습 정답 레이블 2    20
1    10
Name: label, dtype: int64
학습 정답 레이블 0    50
1    50
2    20
Name: label, dtype: int64
학습 정답 레이블 2    30
Name: label, dtype: int64


In [72]:
#Skf
skf_iris = StratifiedKFold(n_splits=3)
n_iter=0

for train_idx, test_idx in skf_iris.split(iris_df,iris_df['label']):
    n_iter +=1
    label_train = iris_df['label'].iloc[train_idx]
    label_test = iris_df['label'].iloc[test_idx]
    print('학습 정답 레이블', label_train.value_counts())
    print('학습 정답 레이블', label_test.value_counts())

학습 정답 레이블 2    34
0    33
1    33
Name: label, dtype: int64
학습 정답 레이블 0    17
1    17
2    16
Name: label, dtype: int64
학습 정답 레이블 1    34
0    33
2    33
Name: label, dtype: int64
학습 정답 레이블 0    17
2    17
1    16
Name: label, dtype: int64
학습 정답 레이블 0    34
1    33
2    33
Name: label, dtype: int64
학습 정답 레이블 1    17
2    17
0    16
Name: label, dtype: int64


- y값의 희소성으로 발생할 수 있는 문제 : 사기탐지, 이상치 탐지 (공모전 주 주제)
- y값이 희소한 경우 대부분의 거래는 정상이지만 비정상이 있을 수 있다.(비정상이 낮은 경우)
- 즉, 클래스 불균형이 생길 수 있다.
- 대부분 문제들이 예측을 하는 경우에는 모든 상황이 0,1 이진으로 나눌 때 균형잡히기가 어려울 수 있다.
- 이런 부분을 학습할 때 중요한 포인트이니 학습시에 꼭 불균형을 체크해야한다!